# Boto3

## Connect

In [8]:
import boto3

client = boto3.client(
    's3',
    endpoint_url='https://s3.hpccloud.lngs.infn.it/',
    aws_access_key_id='JIUG9PGAKP2ULWP3WQ22',
    aws_secret_access_key='VLynhYvsV9JXGAqVxAMWdmo7X1THDB2p2lwxX7uy',
)

## List all in a bucket

In [16]:
bucket_name = 'geolander.streetview'

In [17]:
paginator = client.get_paginator('list_objects_v2')
for page in paginator.paginate(Bucket=bucket_name, Prefix=''):
    for obj in page.get('Contents', []):
        print(obj['Key'])

2025/azno1/laser/1/Run 12239_20250605_083706_0001.las
2025/azno1/laser/1/Run 12239_20250605_083706_0001.las.txt
2025/azno1/laser/1/Run 12240_20250605_083706_0001.las
2025/azno1/laser/1/Run 12240_20250605_083706_0001.las.txt
2025/azno1/laser/10/Run 102239_20250605_090358_0010.las
2025/azno1/laser/10/Run 102239_20250605_090358_0010.las.txt
2025/azno1/laser/10/Run 102240_20250605_090358_0010.las
2025/azno1/laser/10/Run 102240_20250605_090358_0010.las.txt
2025/azno1/laser/100/Run 1002239_20250605_123118_0100.las
2025/azno1/laser/100/Run 1002239_20250605_123118_0100.las.txt
2025/azno1/laser/100/Run 1002240_20250605_123118_0100.las
2025/azno1/laser/100/Run 1002240_20250605_123118_0100.las.txt
2025/azno1/laser/101/Run 1012239_20250605_123157_0101.las
2025/azno1/laser/101/Run 1012239_20250605_123157_0101.las.txt
2025/azno1/laser/101/Run 1012240_20250605_123157_0101.las
2025/azno1/laser/101/Run 1012240_20250605_123157_0101.las.txt
2025/azno1/laser/102/Run 1022239_20250605_123337_0102.las
2025/a

KeyboardInterrupt: 

## Read a file

### Geo Dataframe

In [14]:
import geopandas as gpd
from io import BytesIO

bucket_name = 'ifab.data'
file_key = 'locations/addresses_IFAB.geojson'

response = client.get_object(Bucket=bucket_name, Key=file_key)
gdf = gpd.read_file(BytesIO(response['Body'].read()))

In [15]:
gdf

,geometry
0,POINT (677211.797 4933317.628)
1,POINT (677241.759 4933385.144)
2,POINT (677293.276 4933510.798)
3,POINT (677319.353 4933571.794)
4,POINT (677359.004 4933488.394)
...,...
7631,POINT (685737.631 4929834.475)
7632,POINT (687085.21 4931160.156)
7633,POINT (692118.907 4930227.227)
7634,POINT (686610.723 4928675.531)


### Image

In [ ]:
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt

bucket_name = 'prova.ifab'
file_key = 'immagini/output.png'

response = client.get_object(Bucket=bucket_name, Key=file_key)

# Apri con PIL
img = Image.open(BytesIO(response['Body'].read()))

# Mostra con matplotlib
plt.imshow(img)
plt.axis('off')
plt.show()

## Crea bucket

In [ ]:
client.create_bucket(Bucket='data')

## Upload a file

In [ ]:
bucket_name = 'prova.ifab'

### Geo Dataframe

In [ ]:
# Inserisce in memoria il geodaframe
buffer = BytesIO()
gdf.to_file(buffer, driver='GeoJSON')

# Carica sul bucket
buffer.seek(0)
client.upload_fileobj(buffer, bucket_name, 'addresses_IFAB.geojson')

### Image

In [ ]:
from PIL import Image
from io import BytesIO

# Immagine PIL (creata o elaborata)
img = Image.new('RGB', (100, 100), color='red')

# Salva in un buffer
img_buffer = BytesIO()
img.save(img_buffer, format='PNG')  # o 'JPEG', ecc.

# Upload
img_buffer.seek(0)
client.upload_fileobj(img_buffer, bucket_name, 'immagini/output.png')

## List our bucket

In [9]:
client.list_buckets()

{'ResponseMetadata': {'RequestId': 'tx000005ecd537ff9160ad6-00688b2386-228665030-default',
  'HostId': '',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'transfer-encoding': 'chunked',
   'x-amz-request-id': 'tx000005ecd537ff9160ad6-00688b2386-228665030-default',
   'content-type': 'application/xml',
   'server': 'Ceph Object Gateway (squid)',
   'date': 'Thu, 31 Jul 2025 08:04:22 GMT',
   'access-control-allow-origin': '*',
   'access-control-allow-headers': '*',
   'access-control-expose-headers': 'x-amz-server-side-encryption, x-amz-request-id, x-amz-id-2',
   'access-control-allow-credentials': 'true',
   'access-control-allow-methods': 'GET,PUT,POST,DELETE,PATCH,OPTIONS',
   'access-control-max-age': '3600'},
  'RetryAttempts': 0},
 'Buckets': [{'Name': 'cropping',
   'CreationDate': datetime.datetime(2025, 7, 30, 14, 13, 16, 311000, tzinfo=tzutc())},
  {'Name': 'data',
   'CreationDate': datetime.datetime(2025, 7, 30, 14, 20, 23, 450000, tzinfo=tzutc())},
  {'Name': 'geodaframe',
  

## Delete a file

In [ ]:
bucket_name = 'prova.ifab'
file_key = 'immagini/output.png'

client.delete_object(Bucket=bucket_name, Key=file_key)

## Delete a bucket

In [ ]:
client.delete_bucket(Bucket='prova.ifab')

## Images Migration

In [ ]:
from s3_library.S3Client import S3Client
from io import BytesIO
import re

s3_client = S3Client()

for file in s3_client.list_files('geolander.streetview'):
    if '/laser/' not in file:
        print(file)

In [7]:
from s3_library.S3Client import S3Client
from io import BytesIO

s3_client = S3Client()

count = 0
for file in s3_client.list_files('geolander.streetview'):
    
    try:
        if '/laser/' not in file:
            content = s3_client.read_file('geolander.streetview', file)

            buffer = BytesIO(content)
            s3_client.write_object(buffer, 'data', file)

            count += 1
            print('File copied: ', count)
    except Exception as e:
        print(f'Error processing file {file}: {e}')

File copied:  1
File copied:  2
File copied:  3
File copied:  4
File copied:  5
File copied:  6
File copied:  7
File copied:  8
File copied:  9
File copied:  10
File copied:  11
File copied:  12
File copied:  13
File copied:  14
File copied:  15
File copied:  16
File copied:  17
File copied:  18
File copied:  19
File copied:  20
File copied:  21
File copied:  22
File copied:  23
File copied:  24
File copied:  25
File copied:  26
File copied:  27
File copied:  28
File copied:  29
File copied:  30
File copied:  31
File copied:  32
File copied:  33
File copied:  34
File copied:  35
File copied:  36
File copied:  37
File copied:  38
File copied:  39
File copied:  40
File copied:  41
File copied:  42
File copied:  43
File copied:  44
File copied:  45
File copied:  46
File copied:  47
File copied:  48
File copied:  49
File copied:  50
File copied:  51
File copied:  52
File copied:  53
File copied:  54
File copied:  55
File copied:  56
File copied:  57
File copied:  58
File copied:  59
File c

KeyboardInterrupt: 